In [4]:
# # plot the results
# plt.figure(figsize = (10,8))
# plt.plot(x, y, 'b.')
# plt.plot(x, alpha[0]*x + alpha[1], 'r')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.show()

In [20]:
import numpy as np
import statsmodels.api as sm
import torch
import pandas as pd

N = 200 # Trainingszeilen (historical data)
c_new = 4

# x{k in 0..24, i in 0..N} default if k=0 then 1 else round(np.random.uniform(0, 20)) # historical/training data
x = np.array([[ (1 if k == 0 else np.round(np.random.uniform(0, 20))) for k in range(N)] for _ in range(25)]) # historical/training data


# own period: 1st half: own x[7, i]vs x[2, i]| 2nd half: own x[7, i] vs x[2, i+1] 
# comp period: 1st half: com x[2, i-1] vs x[7, i-1] | 2nd half: com x[2, i-1] vs x[7, i] 

for i in range(1, N):
	x[1, i] = x[1, i-1]-x[12, i-1]+x[13, i-1]  # inventory own

	x[2, i] = x[7, i-1]-1 + np.round_(x[5, i-1]/200) if 9 < x[7, i-1] <= 20 else 20  # comp price new
	x[3, i] = x[8, i-1]-1 + np.round_(x[5, i-1]/200) if 5 < x[8, i-1] <= 15 else 15  # comp price used
	x[4, i] = x[9, i-1]-0.5-np.round_(x[5, i-1]/200)if 1 < x[9, i-1] <= 5 else 5  # comp price rebuy
	x[5, i] = x[5, i-1]-x[16, i-1] + x[17, i - 1]  # comp inventory
	x[6, i] = max(0, np.round_(0.8*x[6, i-1])+x[11, i-1]+x[12, i-1]-x[13, i-1] + x[15, i-1] + x[16, i-1]-x[17, i-1])  # resource in use

	x[7, i] = np.round_(np.random.uniform(5, 20)) if i < 20 else x[2, i] - 1 + np.round_(x[1, i]/200)if 9 < x[2, i] <= 18 else 18  # own price new
	x[8, i] = np.round_(np.random.uniform(0, 20)) if i < 20 else x[3, i] - 1 + np.round_(x[1, i]/200)if 5 < x[3, i] <= 12 else 12  # own price used
	x[9, i] = np.round_(np.random.uniform(0, 10)) if i < 20 else x[4, i]-0.5-np.round_(x[1, i]/200)if 1 < x[4, i] <= 5 else 5  # own price rebuy
	x[10, i] = x[1, i]*0.05  # own holding cost
	x[22, i] = x[7, i]-1+np.round_(x[5, i]/200) if 9 < x[7, i] <= 20 else 20  # comp new reaction
	x[23, i] = x[8, i]-1+np.round_(x[5, i]/200) if 5 < x[8, i] <= 15 else 15  # comp used reaction
	x[24, i] = x[9, i]-0.5 - np.round_(x[5, i]/200) if 1 < x[9, i] <= 5 else 5  # comp rebuy reaction
	x[11, i] = np.round_(max(0, np.random.uniform(5, 15)-x[7, i] + x[2, i]/4 + x[22, i]/4))  # own sales new
	x[12, i] = np.round_(min(x[1, i],  max(0, np.random.uniform(5, 15)-x[8, i] + x[3, i]/4 + x[23, i]/4)))  # own sales used
	x[13, i] = np.round_(min(x[6, i]/2, max(0, np.random.uniform(5, 15)+x[9, i] - x[4, i]/4 - x[24, i]/4)))  # own repurchases
	x[14, i] = x[5, i]*0.05  # comp holding cost
	x[15, i] = np.round_(max(0, np.random.uniform(5, 15)-x[2, i] + x[7, i]/4 + x[7, i-1]/4))  # comp sales new
	x[16, i] = np.round_(min(x[5, i],  max(0, np.random.uniform(5, 15)-x[3, i] + x[8, i]/4 + x[8, i-1]/4)))  # comp sales used
	x[17, i] = np.round_(min(x[6, i]/2, max(0, np.random.uniform(5, 15)+x[4, i] - x[9, i]/4 - x[9, i-1]/4)))  # comp repurchases

	x[18, i] = -x[10, i] + x[11, i]*(x[7, i]-c_new) + x[12, i]*x[8, i] - x[13, i]*x[9, i]  # own total rewards
	x[19, i] = -x[14, i] + x[15, i]*(x[2, i]-c_new) + x[16, i]*x[3, i] - x[17, i]*x[4, i]  # comp total rewards
	x[20, i] = x[18, i] + x[20, i-1] if i > 0 else 0  # own total accumulated rewards
	x[21, i] = x[19, i] + x[21, i-1] if i > 0 else 0  # comp total accumulated rewards

print("x:", x)
print("x.size", x.size)

x_df = pd.DataFrame(x)
x_df.to_csv("x_first_prototype.csv")
# for {i in range(1, n):
# x[1, i] = x[1, i-1]-x[12, i-1]+x[13, i-1] # inventory own

# x[2, i] = if 9<x[7, i-1]<=20 then x[7, i-1]-1 +round(x[5, i-1]/200) else 20 # comp price new
# x[3, i] = if 5<x[8, i-1]<=15 then x[8, i-1]-1 +round(x[5, i-1]/200) else 15 # comp price used
# x[4, i] = if 1<x[9, i-1]<= 5 then x[9, i-1]-0.5-round(x[5, i-1]/200) else 5 # comp price rebuy
# x[5, i] = x[5, i-1]-x[16, i-1]+x[17, i-1]# comp inventory
# x[6, i] = max(0, round(0.8*x[6, i-1])+x[11, i-1]+x[12, i-1]-x[13, i-1]+x[15, i-1]+x[16, i-1]-x[17, i-1]) # resource in use
# x[7, i] = if i<20 then round(np.random.uniform(5, 20)) else if 9<x[2, i]<=18 then x[2, i]-1 +round(x[1, i]/200) else 18 # own price new
# x[8, i] = if i<20 then round(np.random.uniform(0, 20)) else if 5<x[3, i]<=12 then x[3, i]-1 +round(x[1, i]/200) else 12 # own price used
# x[9, i] = if i<20 then round(np.random.uniform(0, 10)) else if 1<x[4, i]<= 5 then x[4, i]-0.5-round(x[1, i]/200) else 5 # own price rebuy
# x[10, i]= x[1, i]*0.05# own holding cost
# x[22, i]= if 9<x[7, i]<=20 then x[7, i]-1+round(x[5, i]/200) else 20 # comp new reaction
# x[23, i]= if 5<x[8, i]<=15 then x[8, i]-1+round(x[5, i]/200) else 15 # comp used reaction
# x[24, i]= if 1<x[9, i]<= 5 then x[9, i]-0.5 -round(x[5, i]/200) else 5 # comp rebuy reaction
# x[11, i]= round(max(0, np.random.uniform(5, 15)-x[7, i] +x[2, i]/4 +x[22, i]/4 )) # own sales new
# x[12, i]= round(min(x[1, i],  max(0, np.random.uniform(5, 15)-x[8, i] +x[3, i]/4 +x[23, i]/4))) # own sales used
# x[13, i]= round(min(x[6, i]/2, max(0, np.random.uniform(5, 15)+x[9, i] -x[4, i]/4 -x[24, i]/4))) # own repurchases
# x[14, i]= x[5, i]*0.05# comp holding cost
# x[15, i]= round(max(0, np.random.uniform(5, 15)-x[2, i] +x[7, i]/4 +x[7, i-1]/4 )) # comp sales new
# x[16, i]= round(min(x[5, i],  max(0, np.random.uniform(5, 15)-x[3, i] +x[8, i]/4 +x[8, i-1]/4 ))) # comp sales used
# x[17, i]= round(min(x[6, i]/2, max(0, np.random.uniform(5, 15)+x[4, i] -x[9, i]/4 -x[9, i-1]/4 ))) # comp repurchases

# x[18, i]= -x[10, i] +x[11, i]*(x[7, i]-c_new) +x[12, i]*x[8, i] -x[13, i]*x[9, i] # own total rewards
# x[19, i]= -x[14, i] +x[15, i]*(x[2, i]-c_new) +x[16, i]*x[3, i] -x[17, i]*x[4, i] # comp total rewards
# x[20, i]= x[18, i]+ if i>0 then x[20, i-1] # own total accumulated rewards
# x[21, i]= x[19, i]+ if i>0 then x[21, i-1] # comp total accumulated rewards




x: [[  1.    5.   16.  ...   5.   14.    4. ]
 [  1.    1.    2.  ... 117.  117.  113. ]
 [  1.   20.   12.  ...  19.   19.   19. ]
 ...
 [  1.   12.   15.  ...  19.   19.   19. ]
 [  1.    7.   15.  ...  13.   13.   13. ]
 [  1.    5.    1.5 ...   1.    2.5   5. ]]
x.size 5000


In [6]:
########  OLS y1 (own sales new)

y1 = [x[11,i] for i in range(N)]
# print("y1:", y1)
y1_df = pd.DataFrame(y1)
y1_df.to_csv("y1_first_prototype.csv")
M1 = (0,1,2,3,4,7,8,9,22,23,24)
# M1: 0-0;1-1;2-2;3-3;4-4;7-5;8-6;9-7;22-8;23-9;24-10
# param b1 {k in M1} default 0
b1 = np.zeros(len(M1))
# var beta1 {k in M1}
# beta1 = somehow do the beta error
x_y1 = x[M1,:]

# print(x_y1)
# minimize OLSy1: sum{i in 1..N} ( sum{k in M1} beta1[k]*x[k,i] - y1[i] )^2
# ols_y1 = np.sum((np.dot(b1[M1], x[M1,:]) - y1)**2) #########################################################################
# y1_t = torch.tensor(y1)

# y_y1_t = torch.tensor(x_y1
# print(y1_t.dim())
# x_y1i = torch.transpose(y1_t, -1, 0)
# y1_t1 = y1_t[:, np.newaxis]


# print("y1_t:", y1_t.size())
# print("x_y1:", x_y1)
# print("alpha:", alpha)
transposed_tensor_x_y1 = torch.tensor(x_y1).transpose(0,1)
tensor_y1 = torch.tensor(y1)
result_tuple_y1 = torch.linalg.lstsq(transposed_tensor_x_y1, tensor_y1, driver='gels')
print("result:", result_tuple_y1)
# objective OLSy1 solve for{k in M1}   b1[k] = beta1[k]

b1 = result_tuple_y1[0]
print(torch.dist(torch.dot(b1, transposed_tensor_x_y1[0]), tensor_y1[0]))


result: torch.return_types.linalg_lstsq(
solution=tensor([ 0.0215, -0.0015,  0.2137, -0.0289,  0.3675, -0.6704,  0.1365, -0.0054,
         0.5103, -0.1924,  0.4425], dtype=torch.float64),
residuals=tensor([1419.0952], dtype=torch.float64),
rank=tensor([], dtype=torch.int64),
singular_values=tensor([], dtype=torch.float64))
tensor(0.2066, dtype=torch.float64)


In [17]:
########  OLS y2 (own sales used)

y2 = [x[12,i] for i in range(N)]
M2 = (0,1,2,3,4,7,8,9,22,23,24)
x_y2 = x[M2,:]

# print("y2:", y2)
# y2_df = pd.DataFrame(y2)
# y2_df.to_csv("y2_first_prototype.csv")
# param b1 {k in M1} default 0
# b2 = np.zeros(len(M2))
# var beta1 {k in M1}
# beta1 = somehow do the beta error

# print("x_y2:", x_y2)


# param y2 {i in 1..N} := x[12,i];
# set M2 := {0,1,2,3,4,7,8,9,22,23,24};
# param b2 {k in M2} default 0;
# var beta2 {k in M2};

transposed_tensor_x_y2 = torch.tensor(x_y2).transpose(0,1)
tensor_y2 = torch.tensor(y2)
result_tuple_y2 = torch.linalg.lstsq(transposed_tensor_x_y2, tensor_y2)
print("result:", result_tuple_y2)
b2 = result_tuple_y2[0]

# minimize OLSy2: sum{i in 1..N} ( sum{k in M2} beta2[k]*x[k,i] - y2[i] )^2;
# objective OLSy2; solve; for{k in M2} let b2[k]:=beta2[k];;

result: torch.return_types.linalg_lstsq(
solution=tensor([ 0.0540,  0.0036,  0.2151,  0.0978,  0.6240,  0.0932, -0.5012,  0.2058,
         0.1057, -0.2769,  0.3789], dtype=torch.float64),
residuals=tensor([], dtype=torch.float64),
rank=tensor(11),
singular_values=tensor([], dtype=torch.float64))


In [34]:
########  OLS y3 (own purchases rebuy)

y3 = [x[13,i] for i in range(N)]
M3 = (0,1,2,3,4,7,8,9,22,23,24)
x_y3 = x[M3,:]

# param y3 {i in 1..N} := x[13,i];
# set M3 := {0,1,2,3,4,7,8,9,22,23,24};
# param b3 {k in M3} default 0;
# var beta3 {k in M3};

transposed_tensor_x_y3 = torch.tensor(x_y3).transpose(0,1)
tensor_y3 = torch.tensor(y3)

print("x_y4:", torch.tensor(x_y3).size())
print("y3:", tensor_y3.size())
print("transposed_tensor_x_y3:", transposed_tensor_x_y3.size())

result_tuple_y3 = torch.linalg.lstsq(transposed_tensor_x_y3, tensor_y3)
print("result:", result_tuple_y3)
b3 = result_tuple_y3[0]
# minimize OLSy3: sum{i in 1..N} ( sum{k in M3} beta3[k]*x[k,i] - y3[i] )^2;
# objective OLSy3; solve; for{k in M3} let b3[k]:=beta3[k];;

x_y4: torch.Size([11, 200])
y3: torch.Size([200])
transposed_tensor_x_y3: torch.Size([200, 11])
result: torch.return_types.linalg_lstsq(
solution=tensor([-0.0219, -0.0128,  0.1852, -0.0882,  0.5628, -0.2588,  0.0932,  0.3274,
         0.1621,  0.1010,  0.5221], dtype=torch.float64),
residuals=tensor([], dtype=torch.float64),
rank=tensor(11),
singular_values=tensor([], dtype=torch.float64))


In [56]:
########  OLS y4 (price comp new)

y4 = [x[2,i+1] for i in range(0, N-1)]
M4 = (0,2,3,4,7,8,9)
M4x = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20)

x_y4_x = [[1 if x[7,i] < k else 0 for k in M4x] for i in range(1, N) ] # matrix for price own new (the 0 and 1 stuff)

x_y4 = x[M4,:(len(x[0]) - 1)]
print(len(x[0]))
# print("x_y4:", x_y4)
# print("x_y4_x:", x_y4_x)
x_y4_tensor = torch.tensor(x_y4)
x_y4_x_tensor = torch.tensor(x_y4_x)
tensor_y4 = torch.tensor(y4)
x_y4_combined = torch.concat((x_y4_tensor, x_y4_x_tensor.transpose(0, 1)), 0)

print("x_y4:", x_y4_tensor.size())
print("x_y4_x:", x_y4_x_tensor.size())
print("y4:", tensor_y4.size())
print("x_y4_combined:", x_y4_combined.size())

result_tuple_y4 = torch.linalg.lstsq(x_y4_combined.transpose(0, 1), tensor_y4)
print("result:", result_tuple_y4)
b4_b4x = result_tuple_y4[0]

b4 = b4_b4x[0:len(M4)]
b4x = b4_b4x[len(M4):]
print("b4:", b4)
print("b4x:", b4x)
# param y4 {i in 1..N-1} := x[2,i+1]; # x22
# set M4 := {0,2,3,4,7,8,9};
# set M4x:= 1..20;
# param x4 {k in M4x,i in 1..N-1} := if x[7,i]<k then 1 else 0;
# param b4 {k in M4} default 0;
# param b4x{k in M4x} default 0;
# var beta4 {k in M4};
# var beta4x{k in M4x};

# minimize OLSy4: sum{i in 1..N-1} ( sum{k in M4}  beta4[k] *x[k,i] 
#                                  + sum{k in M4x} beta4x[k]*x4[k,i] - y4[i] )^2;
# objective OLSy4; solve; for{k in M4}  let b4[k]:=beta4[k];;
#                         for{k in M4x} let b4x[k]:=beta4x[k];;

200
x_y4: torch.Size([7, 199])
x_y4_x: torch.Size([199, 20])
y4: torch.Size([199])
x_y4_combined: torch.Size([27, 199])
result: torch.return_types.linalg_lstsq(
solution=tensor([ 0.0510,  0.1960,  0.2683, -0.0633,  0.6541,  0.0155, -0.1617,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0027,
        -0.0034, -0.0165, -0.0204, -0.0379, -0.0376, -0.0385, -0.0360, -0.0393,
        -0.0461,  0.0158,  0.0158], dtype=torch.float64),
residuals=tensor([], dtype=torch.float64),
rank=tensor(7),
singular_values=tensor([], dtype=torch.float64))
b4: tensor([ 0.0510,  0.1960,  0.2683, -0.0633,  0.6541,  0.0155, -0.1617],
       dtype=torch.float64)
b4x: tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0027, -0.0034, -0.0165, -0.0204, -0.0379, -0.0376, -0.0385, -0.0360,
        -0.0393, -0.0461,  0.0158,  0.0158], dtype=torch.float64)


In [57]:
########  OLS y5 (price comp used)

y5 = [x[3,i+1] for i in range(0, N-1)]
M5 = (0,2,3,4,7,8,9)
M5x = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20)

x_y5_x = [[1 if x[8,i] < k else 0 for k in M5x] for i in range(1, N) ] # matrix for price own new (the 0 and 1 stuff)

x_y5 = x[M5,:(len(x[0]) - 1)]

x_y5_tensor = torch.tensor(x_y5)
x_y5_x_tensor = torch.tensor(x_y5_x)
tensor_y5 = torch.tensor(y5)
x_y5_combined = torch.concat((x_y5_tensor, x_y5_x_tensor.transpose(0, 1)), 0)

print("x_y5:", x_y5_tensor.size())
print("x_y5_x:", x_y5_x_tensor.size())
print("y5:", tensor_y5.size())
print("x_y5_combined:", x_y5_combined.size())

result_tuple_y5 = torch.linalg.lstsq(x_y5_combined.transpose(0, 1), tensor_y5)
print("result:", result_tuple_y5)
print("result:", result_tuple_y4)
b5_b5x = result_tuple_y5[0]

b5 = b5_b5x[0:len(M5)]
b5x = b5_b5x[len(M5):]
print("b5:", b5)
print("b5x:", b5x)

# param y5 {i in 1..N-1} := x[3,i+1];
# set M5 := {0,2,3,4,7,8,9};
# set M5x:= 1..20;
# param x5 {k in M5x,i in 1..N-1} := if x[8,i]<k then 1 else 0;
# param b5 {k in M5} default 0;
# param b5x{k in M5x} default 0;
# var beta5 {k in M5};
# var beta5x{k in M5x};

# minimize OLSy5: sum{i in 1..N-1} ( sum{k in M5}  beta5[k] *x[k,i] 
#                                  + sum{k in M5x} beta5x[k]*x5[k,i] - y5[i] )^2;
# objective OLSy5; solve; for{k in M5}  let b5[k]:=beta5[k];;
#                         for{k in M5x} let b5x[k]:=beta5x[k];;

x_y5: torch.Size([7, 199])
x_y5_x: torch.Size([199, 20])
y5: torch.Size([199])
x_y5_combined: torch.Size([27, 199])
result: torch.return_types.linalg_lstsq(
solution=tensor([ 0.0846,  0.0733,  0.3778,  0.4318,  0.2647,  0.1770,  0.5011, -2.1910,
        -3.5578,  0.5906,  0.5906, -0.0325,  0.2197,  0.3778,  0.4170, -0.3621,
        -0.5070, -2.4835, -1.6863, -0.8440, -0.5640,  0.0319,  0.0319, -0.6648,
        -0.1486, -0.1486, -0.1486], dtype=torch.float64),
residuals=tensor([], dtype=torch.float64),
rank=tensor(10),
singular_values=tensor([], dtype=torch.float64))
result: torch.return_types.linalg_lstsq(
solution=tensor([ 0.0510,  0.1960,  0.2683, -0.0633,  0.6541,  0.0155, -0.1617,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0027,
        -0.0034, -0.0165, -0.0204, -0.0379, -0.0376, -0.0385, -0.0360, -0.0393,
        -0.0461,  0.0158,  0.0158], dtype=torch.float64),
residuals=tensor([], dtype=torch.float64),
rank=tensor(7),
singular_values=tens

In [58]:
y6 = [x[4,i+1] for i in range(0, N-1)]
M6 = (0,2,3,4,7,8,9)
M6x = (1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10)

x_y6_x = [[1 if x[9,i] < k else 0 for k in M6x] for i in range(1, N) ] # matrix for price own new (the 0 and 1 stuff)

x_y6 = x[M6,:(len(x[0]) - 1)]

x_y6_tensor = torch.tensor(x_y6)
x_y6_x_tensor = torch.tensor(x_y6_x)
tensor_y6 = torch.tensor(y6)
x_y6_combined = torch.concat((x_y6_tensor, x_y6_x_tensor.transpose(0, 1)), 0)

print("x_y6:", x_y6_tensor.size())
print("x_y6_x:", x_y6_x_tensor.size())
print("y6:", tensor_y6.size())
print("x_y6_combined:", x_y6_combined.size())

result_tuple_y6 = torch.linalg.lstsq(x_y6_combined.transpose(0, 1), tensor_y6)
print("result:", result_tuple_y6)
print("result:", result_tuple_y4)
b6_b6x = result_tuple_y6[0]

b6 = b6_b6x[0:len(M6)]
b6x = b6_b6x[len(M6):]
print("b6:", b6)
print("b6x:", b6x)

x_y6: torch.Size([7, 199])
x_y6_x: torch.Size([199, 19])
y6: torch.Size([199])
x_y6_combined: torch.Size([26, 199])
result: torch.return_types.linalg_lstsq(
solution=tensor([ 8.0793e-03,  1.4096e-01,  2.7969e-02, -7.3982e-03, -1.5354e-01,
         4.3933e-03,  3.0275e-01, -2.9870e-01, -5.3024e-01,  4.4837e-02,
        -3.0968e-01, -3.5838e-01, -2.5917e+00,  9.4329e-01, -8.7638e-01,
         4.6521e+00, -1.6371e+00, -1.6371e+00, -2.8470e-01, -2.8470e-01,
         5.0413e-01,  5.0413e-01,  6.4776e-01,  6.4776e-01,  5.3141e-01,
         5.3141e-01], dtype=torch.float64),
residuals=tensor([], dtype=torch.float64),
rank=tensor(17),
singular_values=tensor([], dtype=torch.float64))
result: torch.return_types.linalg_lstsq(
solution=tensor([ 0.0510,  0.1960,  0.2683, -0.0633,  0.6541,  0.0155, -0.1617,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0027,
        -0.0034, -0.0165, -0.0204, -0.0379, -0.0376, -0.0385, -0.0360, -0.0393,
        -0.0461,  0.0158, 

In [69]:
###########  Simulation in Modell B   ###############

NB = N

# param xb{k in 0..24,i in 0..NB} default if k=0 then 1 else if i=0 then 5 else -1      # simulated data
xb = np.array([[(1 if k == 0 else 5 if i == 0 else -1) for k in range(0, 24)] for i in range(0, NB)])

for i in range(1, NB):
	xb[1, i] = xb[1, i-1]-xb[12, i-1]+xb[13, i-1]

	# xb[2,i] = sum{k in M4}  b4[k] *xb[k,i-1] + sum{k in M4x} (b4x[k]*(1 if xb[7,i-1]<k else 0)) for k in M4x
	# xb[2,i] = [(b4[k] *xb[k,i-1] + (b4x[k2]*(1 if xb[7,i-1]<k2 else 0)) for k2 in M4x) for k in M4]  # comp price new
	for ki, k in enumerate(M4):
		print("k:", k, "ki:", ki)
		xb[2, i] += b4[ki] * xb[k, i-1]
	for k, ki in enumerate(M4x):
		xb[2, i] += (b4x[ki] * (1 if xb[7, i-1] < k else 0))  # comp price new

	# xb[3,i] = sum{k in M5}  b5[k] *xb[k,i-1] + sum{k in M5x} b5x[k]*(if xb[8,i-1]<k then 1 else 0)  # comp price used
	for k, i in enumerate(M5):
		xb[3, i] += b5[ki] * xb[k, i-1]
	for k, i in enumerate(M5x):
		xb[3, i] += b5x[k]*(1 if xb[8, i-1] < k else 0)  # comp price used


	xb[4, i] = sum([b6[ki] * xb[k, i-1] for k in M6]) + sum([b6x[ki]*(1 if xb[9, i-1] < k else 0) for k, ki in enumerate(M6x)])
	# xb[4,i] = sum{k in M6}  b6[k] *xb[k,i-1] + sum{k in M6x} b6x[k]*(if xb[9,i-1]<k then 1 else 0)  # comp price rebuy

	xb[5, i] = xb[5, i-1] - xb[16, i-1] + xb[17, i-1]  # comp inventory

	xb[6, i] = max(0, np.round_(0.8*xb[6, i-1]) + xb[11, i-1] + xb[12, i-1] - xb[13, i-1] + xb[15, i-1] + xb[16, i-1] - xb[17, i-1])  # resources in use

	xb[7, i] = xb[2, i] - 1 + np.round_(x[1, i]/200) if 9 < xb[2, i] <= 18 else 18  # own price new
	xb[8, i] = xb[3, i] - 1 + np.round_(x[1, i]/200) if 5 < xb[3, i] <= 12 else 12  # own price used
	xb[9, i] = xb[4, i] - 0.5 - np.round_(x[1, i]/200) if 1 < xb[4, i] <= 5 else 5  # own price rebuy
	xb[10, i] = xb[1, i] * 0.05  # own holding cost

	# xb[22,i]= sum{k in M4}  b4[k] *xb[k,i] + sum{k in M4x} b4x[k]*(if xb[7,i]<k then 1 else 0)  # comp new reaction
	for k, ki in enumerate(M4):
		xb[22, i] += b4[ki] * xb[k, i]
	for k, ki in enumerate(M4x):
		b4x[ki] * (1 if xb[7, i] < k else 0)

	# xb[23,i]= sum{k in M5}  b5[k] *xb[k,i] + sum{k in M5x} b5x[k]*(if xb[8,i]<k then 1 else 0)  # comp used reaction
	for k, ki in enumerate(M5):
		xb[23, i] += b5[k] * xb[k, i]
	for k, ki in enumerate(M5x):
		xb[23, i] += b5x[k]*(1 if xb[8, i] < k else 0)

	# xb[24,i]= sum{k in M6}  b6[k] *xb[k,i] + sum{k in M6x} b6x[k]*(if xb[9,i]<k then 1 else 0)  # comp rebuy reaction
	for k, ki in enumerate(M6):
		xb[24, i] += b6[k] * xb[k, i]
	for k, ki in enumerate(M6x):
		xb[24, i] += b6x[k]*(1 if xb[9, i] < k else 0)

	# xb[11,i]= np.round_(max(0, np.random.uniform(-5,5) + sum{k in M1} b1[k]*xb[k,i]))
	xb[11, i] = np.round_(max(np.random.uniform(-5, 5) + sum([b1[k] * xb[k, i] for k, ki in enumerate(M1)])))

	# xb[12,i]= np.round_(min(x[1,i],  max(0,np.random.uniform(-5,5) + sum{k in M2} b2[k]*xb[k,i])))
	xb[12, i] = np.round_(min(x[1, i], max(np.random.uniform(-5, 5) + sum([b2[ki] * xb[k, i] for k, ki in enumerate(M2)]), 0)))
	# xb[13,i]= np.round_(min(x[6,i]/2,max(0,np.random.uniform(-5,5) + sum{k in M3} b3[k]*xb[k,i])))
	xb[13, i] = np.round_(min(x[6, i] / 2, max(np.random.uniform(-5, 5) + sum([b3[ki] * xb[k, i] for k, ki in enumerate(M3)]), 0)))
	xb[14, i] = xb[5, i]*0.05

	xb[15, i] = np.round_(max(0, np.random.uniform(-5, 5) + b1[0] * xb[0, i] + b1[1] * xb[5, i] + b1[2] * xb[7, i-1] + b1[3] * xb[8, i-1]
		+ b1[4] * xb[9, i-1] + b1[7] * xb[2, i] + b1[8] * xb[3, i] + b1[9] * xb[4, i] + b1[22] * xb[7, i] + b1[23] * xb[8, i]
		+ b1[24] * xb[9, i]))  # cf M1

	xb[16, i] = np.round_(min(x[5, i],  max(0, np.random.uniform(-5, 5) + b2[0] * xb[0, i] + b2[1] * xb[5, i] + b2[2] * xb[7, i-1]
		+ b2[3] * xb[8, i - 1]
		+ b2[4] * xb[9, i-1] + b2[7] * xb[2, i] + b2[8] * xb[3, i] + b2[9] * xb[4, i] + b2[22] * xb[7, i] + b2[23] * xb[8, i]
		+ b2[24] * xb[9, i])))  # cf M2

	xb[17, i] = np.round_(min(x[6, i]/2, max(0, np.np.random.uniform(-5, 5) + b3[0] * xb[0, i] + b3[1] * xb[5, i] + b3[2] * xb[7, i-1]
		+ b3[3] * xb[8, i-1] + b3[4] * xb[9, i-1] + b3[7] * xb[2, i] + b3[8] * xb[3, i] + b3[9] * xb[4, i] + b3[22] * xb[7, i]
		+ b3[23] * xb[8, i] + b3[24] * xb[9, i])))  # cf M3

	xb[18, i] = -xb[10, i]+xb[11, i]*(xb[7, i] - c_new)+xb[12, i]*xb[8, i]-xb[13, i]*xb[9, i]      # own total rewards
	xb[19, i] = -xb[14, i]+xb[15, i]*(xb[2, i] - c_new)+xb[16, i]*xb[3, i]-xb[17, i]*xb[4, i]      # comp total rewards
	xb[20, i] = xb[18, i] + (xb[20, i-1] if i > 0 else 0)  # own total accumulated rewards
	xb[21, i] = xb[19, i] + (xb[21, i-1] if i > 0 else 0)  # comp total accumulated rewards


k: 0 ki: 0
k: 2 ki: 1
k: 3 ki: 2
k: 4 ki: 3
k: 7 ki: 4
k: 8 ki: 5
k: 9 ki: 6


IndexError: index 20 is out of bounds for dimension 0 with size 20

In [101]:
###########  Simulation in Modell B   ###############

import numpy as np
NB = N

# param xb{k in 0..24,i in 0..NB} default if k=0 then 1 else if i=0 then 5 else -1      # simulated data
xb = np.array([[(1 if k-1 == 0 else 5 if i == 0 else -1) for k in range(0, 25)] for i in range(0, NB)]).transpose()
print(xb.shape[0])
for i in range(1, NB):
	xb[1, i] = xb[1, i-1]-xb[12, i-1]+xb[13, i-1]

	# xb[2,i] = sum{k in M4}  b4[k] *xb[k,i-1] + sum{k in M4x} (b4x[k]*(1 if xb[7,i-1]<k else 0)) for k in M4x
	# xb[2,i] = [(b4[k] *xb[k,i-1] + (b4x[k2]*(1 if xb[7,i-1]<k2 else 0)) for k2 in M4x) for k in M4]  # comp price new
	for ki, k in enumerate(M4):
		xb[2, i] += b4[ki] * xb[ki, k-1]
	for ki, k in enumerate(M4x):
		xb[2, i] += (b4x[ki] * (1 if xb[7, i-1] < k else 0))  # comp price new

	# xb[3,i] = sum{k in M5}  b5[k] *xb[k,i-1] + sum{k in M5x} b5x[k]*(if xb[8,i-1]<k then 1 else 0)  # comp price used
	for ki, k in enumerate(M5):
		xb[3, i] += b5[ki] * xb[ki, k-1]
	for ki, k in enumerate(M5x):
		xb[3, i] += b5x[ki]*(1 if xb[8, i-1] < k else 0)  # comp price used


	xb[4, i] = sum([b6[ki] * xb[ki, k-1] for ki, k in enumerate(M6)]) + sum([b6x[ki]*(1 if xb[9, i-1] < k else 0) for ki, k in enumerate(M6x)])
	# xb[4,i] = sum{k in M6}  b6[k] *xb[k,i-1] + sum{k in M6x} b6x[k]*(if xb[9,i-1]<k then 1 else 0)  # comp price rebuy

	xb[5, i] = xb[5, i-1] - xb[16, i-1] + xb[17, i-1]  # comp inventory

	xb[6, i] = max(0, np.round_(0.8*xb[6, i-1]) + xb[11, i-1] + xb[12, i-1] - xb[13, i-1] + xb[15, i-1] + xb[16, i-1] - xb[17, i-1])  # resources in use

	xb[7, i] = xb[2, i] - 1 + np.round_(x[1, i]/200) if 9 < xb[2, i] <= 18 else 18  # own price new
	xb[8, i] = xb[3, i] - 1 + np.round_(x[1, i]/200) if 5 < xb[3, i] <= 12 else 12  # own price used
	xb[9, i] = xb[4, i] - 0.5 - np.round_(x[1, i]/200) if 1 < xb[4, i] <= 5 else 5  # own price rebuy
	xb[10, i] = xb[1, i] * 0.05  # own holding cost

	# xb[22,i]= sum{k in M4}  b4[k] *xb[k,i] + sum{k in M4x} b4x[k]*(if xb[7,i]<k then 1 else 0)  # comp new reaction
	for ki, k in enumerate(M4):
		xb[22, i] += b4[ki] * xb[ki, i]
	for ki, k in enumerate(M4x):
		b4x[ki] * (1 if xb[7, i] < k else 0)

	# xb[23,i]= sum{k in M5}  b5[k] *xb[k,i] + sum{k in M5x} b5x[k]*(if xb[8,i]<k then 1 else 0)  # comp used reaction
	for ki, k in enumerate(M5):
		xb[23, i] += b5[ki] * xb[k, i]
	for ki, k in enumerate(M5x):
		xb[23, i] += b5x[ki] * (1 if xb[8, i] < k else 0)
	# xb[24,i]= sum{k in M6}  b6[k] *xb[k,i] + sum{k in M6x} b6x[k]*(if xb[9,i]<k then 1 else 0)  # comp rebuy reaction
	for ki, k in enumerate(M6):
		xb[24, i] += b6[ki] * xb[k, i]
	for ki, k in enumerate(M6x):
		xb[24, i] += b6x[ki]*(1 if xb[9, i] < k else 0)

	# xb[11,i]= np.round_(max(0, np.random.uniform(-5,5) + sum{k in M1} b1[k]*xb[k,i]))
	xb[11, i] = np.round_(max(np.random.uniform(-5, 5) + sum([b1[ki] * xb[k, i] for ki, k in enumerate(M1)]), 0))

	# xb[12,i]= np.round_(min(x[1,i],  max(0,np.random.uniform(-5,5) + sum{k in M2} b2[k]*xb[k,i])))
	xb[12, i] = np.round_(min(x[1, i], max(np.random.uniform(-5, 5) + sum([b2[ki] * xb[k, i] for ki, k in enumerate(M2)]), 0)))
	# xb[13,i]= np.round_(min(x[6,i]/2,max(0,np.random.uniform(-5,5) + sum{k in M3} b3[k]*xb[k,i])))
	xb[13, i] = np.round_(min(x[6, i] / 2, max(np.random.uniform(-5, 5) + sum([b3[ki] * xb[k, i] for ki, k in enumerate(M3)]), 0)))
	xb[14, i] = xb[5, i]*0.05
	# M1: 0-0;1-1;2-2;3-3;4-4;7-5;8-6;9-7;22-8;23-9;24-10
	xb[15, i] = np.round_(max(0, np.random.uniform(-5, 5) + b1[0] * xb[0, i] + b1[1] * xb[5, i] + b1[2] * xb[7, i-1] + b1[3] * xb[8, i-1]
		+ b1[4] * xb[9, i-1] + b1[5] * xb[2, i] + b1[6] * xb[3, i] + b1[7] * xb[4, i] + b1[8] * xb[7, i] + b1[9] * xb[8, i]
		+ b1[10] * xb[9, i]))  # cf M1

	xb[16, i] = np.round_(min(x[5, i],  max(0, np.random.uniform(-5, 5) + b2[0] * xb[0, i] + b2[1] * xb[5, i] + b2[2] * xb[7, i-1]
		+ b2[3] * xb[8, i - 1]
		+ b2[4] * xb[9, i-1] + b2[5] * xb[2, i] + b2[6] * xb[3, i] + b2[7] * xb[4, i] + b2[8] * xb[7, i] + b2[9] * xb[8, i]
		+ b2[10] * xb[9, i])))  # cf M2

	xb[17, i] = np.round_(min(x[6, i]/2, max(0, np.random.uniform(-5, 5) + b3[0] * xb[0, i] + b3[1] * xb[5, i] + b3[2] * xb[7, i-1]
		+ b3[3] * xb[8, i-1] + b3[4] * xb[9, i-1] + b3[5] * xb[2, i] + b3[6] * xb[3, i] + b3[7] * xb[4, i] + b3[8] * xb[7, i]
		+ b3[9] * xb[8, i] + b3[10] * xb[9, i])))  # cf M3

	xb[18, i] = -xb[10, i]+xb[11, i]*(xb[7, i] - c_new)+xb[12, i]*xb[8, i]-xb[13, i]*xb[9, i]      # own total rewards
	xb[19, i] = -xb[14, i]+xb[15, i]*(xb[2, i] - c_new)+xb[16, i]*xb[3, i]-xb[17, i]*xb[4, i]      # comp total rewards
	xb[20, i] = xb[18, i] + (xb[20, i-1] if i > 0 else 0)  # own total accumulated rewards
	xb[21, i] = xb[19, i] + (xb[21, i-1] if i > 0 else 0)  # comp total accumulated rewards

25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [78]:
print(sum([b1[ki] * xb[k, i] for ki, k in enumerate(M1)]))

tensor(-10.7058, dtype=torch.float64)


<!--NAVIGATION-->
< [16.3 Least Squares Regression Derivation (Multivariable Calculus)](chapter16.03-Least-Squares-Regression-Derivation-Multivariable-Calculus.ipynb)  | [Contents](Index.ipynb) | [16.5 Least Square Regression for Nonlinear Functions](chapter16.05-Least-Square-Regression-for-Nonlinear-Functions.ipynb)    >